In [2]:
import torch.nn as nn

In [5]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [6]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 9,  1,  2, 10,  8,  0,  3,  7,  5]), tensor([4, 6]))

In [7]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [8]:
linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[4.4539],
        [2.5503]], grad_fn=<AddmmBackward>)

In [9]:
linear_model.weight

Parameter containing:
tensor([[0.8498]], requires_grad=True)

In [10]:
linear_model.bias

Parameter containing:
tensor([-0.3306], requires_grad=True)

In [11]:
x = torch.ones(1)
linear_model(x)

tensor([0.5192], grad_fn=<AddBackward0>)

In [12]:
x = torch.ones(10,1)
linear_model(x)

tensor([[0.5192],
        [0.5192],
        [0.5192],
        [0.5192],
        [0.5192],
        [0.5192],
        [0.5192],
        [0.5192],
        [0.5192],
        [0.5192]], grad_fn=<AddmmBackward>)

In [14]:
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1)
t_u = torch.tensor(t_u).unsqueeze(1)

t_u.shape

torch.Size([11, 1])

In [16]:
import torch.optim as optim

In [17]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2)

In [18]:
linear_model.parameters()

<generator object Module.parameters at 0x000001ACEB85C348>

In [19]:

list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.9418]], requires_grad=True),
 Parameter containing:
 tensor([-0.1367], requires_grad=True)]

In [27]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)
        
        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [28]:
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)    

Epoch 1, Training loss 288.0866, Validation loss 99.5094
Epoch 1000, Training loss 3.6458, Validation loss 2.6819
Epoch 2000, Training loss 2.6130, Validation loss 4.4392
Epoch 3000, Training loss 2.5796, Validation loss 4.9075

Parameter containing:
tensor([[5.5371]], requires_grad=True)
Parameter containing:
tensor([-18.2191], requires_grad=True)


# 